In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models
from torchvision.models.vgg import VGG
from sklearn.metrics import confusion_matrix
import pandas as pd
import scipy.misc
import random
import sys

if '/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

from matplotlib import pyplot as plt
import numpy as np
import time
import os

## Define FCN16s model for deconvolution layers

In [2]:
class FCN16s(nn.Module):

    def __init__(self, pretrained_net, n_class):
        super().__init__()
        self.n_class = n_class
        self.pretrained_net = pretrained_net
        self.relu    = nn.ReLU(inplace = True)
        self.deconv1 = nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn1     = nn.BatchNorm2d(512)
        self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn2     = nn.BatchNorm2d(256)
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn3     = nn.BatchNorm2d(128)
        self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn4     = nn.BatchNorm2d(64)
        self.deconv5 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn5     = nn.BatchNorm2d(32)
        self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

    def forward(self, x):
        output = self.pretrained_net(x)
        x5 = output['x5']  # size=(N, 512, x.H/32, x.W/32)
        x4 = output['x4']  # size=(N, 512, x.H/16, x.W/16)

        score = self.relu(self.deconv1(x5))               # size=(N, 512, x.H/16, x.W/16)
        score = self.bn1(score + x4)                      # element-wise add, size=(N, 512, x.H/16, x.W/16)
        score = self.bn2(self.relu(self.deconv2(score)))  # size=(N, 256, x.H/8, x.W/8)
        score = self.bn3(self.relu(self.deconv3(score)))  # size=(N, 128, x.H/4, x.W/4)
        score = self.bn4(self.relu(self.deconv4(score)))  # size=(N, 64, x.H/2, x.W/2)
        score = self.bn5(self.relu(self.deconv5(score)))  # size=(N, 32, x.H, x.W)
        score = self.classifier(score)                    # size=(N, n_class, x.H/1, x.W/1)
        
        return score

## Define VGG16 for convolution layers

In [3]:
class VGGNet(VGG):
    def __init__(self, pretrained=True, model='vgg16', requires_grad=True, remove_fc=True, show_params=False):
        super().__init__(make_layers(cfg[model]))
        self.ranges = ranges[model]

        if pretrained:
            exec("self.load_state_dict(models.%s(pretrained=True).state_dict())" % model)

        if not requires_grad:
            for param in super().parameters():
                param.requires_grad = False

        if remove_fc:  # delete redundant fully-connected layer params, can save memory
            del self.classifier

        if show_params:
            for name, param in self.named_parameters():
                print(name, param.size())

    def forward(self, x):
        output = {}

        # get the output of each maxpooling layer (5 maxpool in VGG net)
        for idx in range(len(self.ranges)):
            for layer in range(self.ranges[idx][0], self.ranges[idx][1]):      
                x = self.features[layer](x)
            output["x%d"%(idx+1)] = x
        return output

In [4]:
ranges = {
    'vgg11': ((0, 3), (3, 6),  (6, 11),  (11, 16), (16, 21)),
    'vgg13': ((0, 5), (5, 10), (10, 15), (15, 20), (20, 25)),
    'vgg16': ((0, 5), (5, 10), (10, 17), (17, 24), (24, 31)),
    'vgg19': ((0, 5), (5, 10), (10, 19), (19, 28), (28, 37))
}

# cropped version from https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py
cfg = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

## Define training parameter

In [5]:
batch_size = 6
epochs     = 50  #500
lr         = 1e-4
momentum   = 0
w_decay    = 1e-5
step_size  = 50
gamma      = 0.5
model_use  = "text_spotting" # "products_20" "mini_competition"
n_class = 16

## Define path, directory trainning environment

In [6]:
# get data
#data_dir  = os.path.join("data", model_use)
data_dir = "data"
if not os.path.exists(data_dir):
    print("Data not found!")
# create dir for model
model_dir = os.path.join("models", model_use)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
# create dir for score
score_dir = os.path.join("scores", model_use)
if not os.path.exists(score_dir):
    os.makedirs(score_dir)

use_gpu = torch.cuda.is_available()
num_gpu = list(range(torch.cuda.device_count()))

vgg_model = VGGNet(requires_grad=True, remove_fc=True)
fcn_model = FCN16s(pretrained_net=vgg_model, n_class=n_class)
#use_gpu = False
if use_gpu:
    ts = time.time()
    vgg_model = vgg_model.cuda()
    fcn_model = fcn_model.cuda()
    fcn_model = nn.DataParallel(fcn_model, device_ids=num_gpu)
    print("Finish cuda loading, time elapsed {}".format(time.time() - ts))

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:48<00:00, 11355035.96it/s]


Finish cuda loading, time elapsed 8.682100772857666


## Visualize model

In [7]:
print(fcn_model)
params = list(fcn_model.parameters())

DataParallel(
  (module): FCN16s(
    (pretrained_net): VGGNet(
      (features): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace)
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace)
        (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (13): ReLU(inplace)
        (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  

Dataset class
-------------

``torch.utils.data.Dataset`` is an abstract class representing a
dataset.
Your custom dataset should inherit ``Dataset`` and override the following
methods:

-  ``__len__`` so that ``len(dataset)`` returns the size of the dataset.
-  ``__getitem__`` to support the indexing such that ``dataset[i]`` can
   be used to get $i$\ th sample

Let's create a dataset class for our face landmarks dataset. We will
read the csv in ``__init__`` but leave the reading of images to
``__getitem__``. This is memory efficient because all the images are not
stored in the memory at once but read as required.



In [13]:
means     = np.array([103.939, 116.779, 123.68]) / 255. # mean of three channels in the order of BGR
h, w      = 480, 640
val_h     = h
val_w     = w
class product_dataset(Dataset):

    def __init__(self, csv_file, phase, n_class=n_class, flip_rate=0.):
        self.data      = pd.read_csv(csv_file)
        self.means     = means
        self.n_class   = n_class
        self.flip_rate = flip_rate
        if phase == 'train':
            self.flip_rate = 0.5

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name   = self.data.iloc[idx, 0]
        img        = cv2.imread(img_name,cv2.IMREAD_UNCHANGED)
        # img        = cv2.imread(os.path.join(data_dir, img_name),cv2.IMREAD_UNCHANGED)
        label_name = self.data.iloc[idx, 1]
        # label      = cv2.imread(os.path.join(data_dir, label_name), cv2.IMREAD_GRAYSCALE)
        label      = cv2.imread(label_name, cv2.IMREAD_GRAYSCALE)
        origin_img = img

        if random.random() < self.flip_rate:
            img   = np.fliplr(img)
            label = np.fliplr(label)

        # reduce mean
        img = img[:, :, ::-1]  # switch to BGR
        
        img = np.transpose(img, (2, 0, 1)) / 255.
        img[0] -= self.means[0]
        img[1] -= self.means[1]
        img[2] -= self.means[2]

        # convert to tensor
        img = torch.from_numpy(img.copy()).float()
        label = torch.from_numpy(label.copy()).long()

        # create one-hot encoding
        h, w = label.size()
        target = torch.zeros(self.n_class, h, w)
        #print(np.unique(label))
        for i in range(self.n_class):
            target[i][label == i] = 1

        sample = {'X': img, 'Y': target, 'l': label, 'origin': origin_img}

        return sample

## Define dataloader and optimizer

In [14]:
# initial dataloader for trainning and validation
# train_file = os.path.join(data_dir, "train.csv")
# val_file   = os.path.join(data_dir, "val.csv")
train_file = "train.csv"
val_file = "val.csv"
train_data = product_dataset(csv_file = train_file, phase = 'train')
val_data   = product_dataset(csv_file = val_file, phase = 'val', flip_rate = 0)
dataloader = DataLoader(train_data, batch_size = batch_size, shuffle=True, num_workers = 0)
val_loader = DataLoader(val_data, batch_size = 4, num_workers = 0)

dataiter = iter(val_loader)

# define loss function
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.RMSprop(fcn_model.parameters(), lr = lr, momentum = momentum, weight_decay = w_decay)
# decay LR by a factor of 0.5 every step_size = 50 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size = step_size, gamma = gamma)  

## Train

In [11]:
def train():
    for epoch in range(epochs):
        fcn_model.train()
        scheduler.step()
        configs    = "FCNs_{}_batch{}_epoch{}_RMSprop_lr{}"\
            .format(model_use, batch_size, epoch, lr)
        model_path = os.path.join(model_dir, configs)
        
        ts = time.time()
        for iter, batch in enumerate(dataloader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = Variable(batch['X'].cuda())
                labels = Variable(batch['Y'].cuda())
            else:
                inputs, labels = Variable(batch['X']), Variable(batch['Y'])

            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch+1, iter, loss.data[0]))
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(fcn_model.state_dict(),model_path + '.pkl')

        val(epoch)

## Evaluation

In [10]:
def val(epoch):
    fcn_model.eval()
    TP = np.zeros(n_class-1)
    FN = np.zeros(n_class-1)
    FP = np.zeros(n_class-1)
    total_ious = []
    pixel_accs = []
    for iter, batch in enumerate(val_loader):
        if use_gpu:
            inputs = Variable(batch['X'].cuda())
        else:
            inputs = Variable(batch['X'])

        output = fcn_model(inputs)
        output = output.data.cpu().numpy()

        N, _, h, w = output.shape
        pred = output.transpose(0, 2, 3, 1).reshape(-1, n_class).argmax(axis=1).reshape(N, h, w)

        target = batch['l'].cpu().numpy().reshape(N, h, w)
        for p, t in zip(pred, target):
            pixel_accs.append(pixel_acc(p, t))
            _TP, _FN, _FP =  analysis(p, t, h, w)
            TP += _TP[1:n_class]
            FN += _FN[1:n_class]
            FP += _FP[1:n_class]
            
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    ious = TP / (TP + FN + FP)
    fscore = 2*TP / (2*TP + FN + FP)
    total_ious = np.array(total_ious).T  # n_class * val_len
    pixel_accs = np.array(pixel_accs).mean()
    
    print("epoch{}, pix_acc: {}, meanIoU: {}, IoUs: {}, recall: {}, precision: {}, fscore: {}"\
          .format(epoch, pixel_accs, np.nanmean(ious), ious, recall, precision, fscore))
    
    f1 = open(score_dir + "/cls_acc_log.txt","a+")
    f1.write('epoch:'+ str(epoch) + ', pix_acc: ' + str(pixel_accs) + '\n' )
    f2 = open(score_dir + "/cls_iou_log.txt","a+")
    f2.write('epoch:'+ str(epoch) + ', class ious: ' + str(ious) + '\n' )
    f3 = open(score_dir + "/mean_iou_log.txt","a+")
    f3.write('epoch:'+ str(epoch) + ', mean IoU: ' + str(np.nanmean(ious)) + '\n' ) 
    f4 = open(score_dir + "/recall_log.txt","a+")
    f4.write('epoch:'+ str(epoch) + ', class recall: ' + str(recall) + '\n' )
    f5 = open(score_dir + "/precision_log.txt","a+")
    f5.write('epoch:'+ str(epoch) + ', class precision: ' + str(precision) + '\n' )    
    f6 = open(score_dir + "/fscore_log.txt","a+")
    f6.write('epoch:'+ str(epoch) + ', class fscore: ' + str(fscore) + '\n' )  
    

def analysis(pred, target, h, w):
    # TP, FN, FP, TN
    TP = np.zeros(n_class)
    FN = np.zeros(n_class)
    FP = np.zeros(n_class)

    target = target.reshape(h * w)
    pred = pred.reshape(h * w)

    con_matrix = confusion_matrix(target, pred,labels = np.arange(0,n_class,1))
    con_matrix[0][0] = 0
    for i in range(0, n_class):
        for j in range(0, n_class):
            if i == j:
                TP[i] += con_matrix[i][j]
            if i != j:
                FP[j] += con_matrix[i][j]
                FN[i] += con_matrix[i][j]
    return TP, FN, FP
                
def pixel_acc(pred, target):
    correct = (pred == target).sum()
    total   = (target == target).sum()
    return correct / total

In [12]:
train()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch1, iter0, loss: 0.7167602181434631
epoch1, iter10, loss: 0.7020950317382812
epoch1, iter20, loss: 0.6988784670829773
epoch1, iter30, loss: 0.6961067914962769
epoch1, iter40, loss: 0.6937254071235657
epoch1, iter50, loss: 0.6912814974784851
epoch1, iter60, loss: 0.6889186501502991
epoch1, iter70, loss: 0.6865101456642151
epoch1, iter80, loss: 0.684277355670929
epoch1, iter90, loss: 0.6820200681686401
epoch1, iter100, loss: 0.6793283820152283
epoch1, iter110, loss: 0.6771360039710999
epoch1, iter120, loss: 0.6742465496063232
epoch1, iter130, loss: 0.6721674203872681
epoch1, iter140, loss: 0.669731616973877
epoch1, iter150, loss: 0.6674121022224426
epoch1, iter160, loss: 0.6649880409240723
epoch1, iter170, loss: 0.6626982688903809
epoch1, iter180, loss: 0.6600777506828308
epoch1, iter190, loss: 0.6581557989120483
epoch1, iter200, loss: 0.6555794477462769
epoch1, iter210, loss: 0.6530529856681824
epoch1, iter220, loss: 0.6508887410163879
epoch1, iter230, loss: 0.6481037735939026
epoch

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in true_divide


epoch0, pix_acc: 0.9865132446823053, meanIoU: 0.012144899491278487, IoUs: [2.78746566e-02 0.00000000e+00 9.57727330e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.18086348e-03 6.27967145e-06
 4.90763375e-02 2.00387695e-02 6.94243215e-02 0.00000000e+00
 9.64232641e-04 3.07581105e-05 0.00000000e+00], recall: [5.76575872e-02 0.00000000e+00 2.51613089e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 6.58620812e-03 6.28824036e-06
 1.54347720e-01 1.09232227e-01 6.41930457e-01 0.00000000e+00
 1.01745230e-03 3.08401631e-05 0.00000000e+00], precision: [0.05120037 0.         0.01522759 0.         0.         0.
 0.02370478 0.00458716 0.06712517 0.02395298 0.07222108        nan
 0.01810051 0.01142857        nan], fscore: [5.42374626e-02 0.00000000e+00 1.89728385e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.03083210e-02 1.25592640e-05
 9.35610418e-02 3.92902115e-02 1.29834940e-01 0.00000000e+00
 1.92660758e-03 6.15143290e-05 0.00000000e+00]
epoch2, iter0, loss: 0.5503596067428589
epoch

epoch4, iter130, loss: 0.16940419375896454
epoch4, iter140, loss: 0.16711577773094177
epoch4, iter150, loss: 0.16524916887283325
epoch4, iter160, loss: 0.16273225843906403
epoch4, iter170, loss: 0.16129182279109955
epoch4, iter180, loss: 0.1588410884141922
epoch4, iter190, loss: 0.15713854134082794
epoch4, iter200, loss: 0.15557821094989777
epoch4, iter210, loss: 0.152631476521492
epoch4, iter220, loss: 0.15085956454277039
epoch4, iter230, loss: 0.14890319108963013
epoch4, iter240, loss: 0.14653557538986206
epoch4, iter250, loss: 0.1443842053413391
epoch4, iter260, loss: 0.14332422614097595
epoch4, iter270, loss: 0.14126484096050262
epoch4, iter280, loss: 0.13960671424865723
epoch4, iter290, loss: 0.1370023787021637
epoch4, iter300, loss: 0.13535211980342865
epoch4, iter310, loss: 0.1334863156080246
epoch4, iter320, loss: 0.13186125457286835
epoch4, iter330, loss: 0.1300349235534668
epoch4, iter340, loss: 0.12891532480716705
epoch4, iter350, loss: 0.12683629989624023
epoch4, iter360, l

...

epoch39, iter360, loss: 0.0005316830356605351
epoch39, iter370, loss: 0.0002564397582318634
epoch39, iter380, loss: 0.000912246061488986
epoch39, iter390, loss: 0.0002502426505088806
epoch39, iter400, loss: 0.0007195628131739795
epoch39, iter410, loss: 0.000394779781345278
epoch39, iter420, loss: 0.000419734074966982
epoch39, iter430, loss: 0.0006750518805347383
epoch39, iter440, loss: 0.00031218797084875405
epoch39, iter450, loss: 0.0006233045132830739
epoch39, iter460, loss: 0.00037252993206493556
epoch39, iter470, loss: 0.0007517659105360508
epoch39, iter480, loss: 0.0004142926773056388
epoch39, iter490, loss: 0.00043158591142855585
epoch39, iter500, loss: 0.0005779266939498484
epoch39, iter510, loss: 0.00028915810980834067
epoch39, iter520, loss: 0.0006904349429532886
epoch39, iter530, loss: 0.0003956687287427485
epoch39, iter540, loss: 0.0005911141051910818
epoch39, iter550, loss: 0.0006304103299044073
epoch39, iter560, loss: 0.0006646651891060174
epoch39, iter570, loss: 0.0014676

epoch42, iter0, loss: 0.0005192614626139402
epoch42, iter10, loss: 0.000699241238180548
epoch42, iter20, loss: 0.0006182183278724551
epoch42, iter30, loss: 0.0011980768758803606
epoch42, iter40, loss: 0.0003379835980013013
epoch42, iter50, loss: 0.0004704413586296141
epoch42, iter60, loss: 0.0003290085296612233
epoch42, iter70, loss: 0.00022683134011458606
epoch42, iter80, loss: 0.0021529512014240026
epoch42, iter90, loss: 0.0005021213437430561
epoch42, iter100, loss: 0.0003972142585553229
epoch42, iter110, loss: 0.0015654682647436857
epoch42, iter120, loss: 0.0003379923873580992
epoch42, iter130, loss: 0.0006637996411882341
epoch42, iter140, loss: 0.0005168399075046182
epoch42, iter150, loss: 0.0005229018861427903
epoch42, iter160, loss: 0.0006854877574369311
epoch42, iter170, loss: 0.000547312549315393
epoch42, iter180, loss: 0.00036058161640539765
epoch42, iter190, loss: 0.00028478424064815044
epoch42, iter200, loss: 0.0009154640138149261
epoch42, iter210, loss: 0.000751136976759880

epoch44, iter270, loss: 0.00024494034005329013
epoch44, iter280, loss: 0.0004903999506495893
epoch44, iter290, loss: 0.0005583124002441764
epoch44, iter300, loss: 0.0010135836200788617
epoch44, iter310, loss: 0.0006872545345686376
epoch44, iter320, loss: 0.0007236694218590856
epoch44, iter330, loss: 0.00024968895013444126
epoch44, iter340, loss: 0.00024263160594273359
epoch44, iter350, loss: 0.0002225116768386215
epoch44, iter360, loss: 0.001956937601789832
epoch44, iter370, loss: 0.00047400270705111325
epoch44, iter380, loss: 0.0004949181457050145
epoch44, iter390, loss: 0.00022879692551214248
epoch44, iter400, loss: 0.000630201306194067
epoch44, iter410, loss: 0.0003274483315180987
epoch44, iter420, loss: 0.0002623645414132625
epoch44, iter430, loss: 0.00035568472230806947
epoch44, iter440, loss: 0.000584438384976238
epoch44, iter450, loss: 0.0005844512488692999
epoch44, iter460, loss: 0.0007631722837686539
epoch44, iter470, loss: 0.0004918791237287223
epoch44, iter480, loss: 0.00036

epoch46, iter540, loss: 0.0007879445911385119
epoch46, iter550, loss: 0.0003557199379429221
epoch46, iter560, loss: 0.0010820544557645917
epoch46, iter570, loss: 0.002000128384679556
Finish epoch 45, time elapsed 468.1316463947296
epoch45, pix_acc: 0.998472803204287, meanIoU: 0.8458348139941937, IoUs: [0.87147591 0.51187242 0.86377836 0.87024193 0.91669891 0.89668715
 0.85865721 0.8984228  0.7509796  0.86535234 0.8703397  0.93604947
 0.71481413 0.92803119 0.93412108], recall: [0.91880105 0.66005934 0.96769501 0.91358437 0.94404493 0.95275676
 0.98240118 0.95495733 0.87895966 0.98541857 0.94906543 0.96262259
 0.96923703 0.97630704 0.96530861], precision: [0.94419456 0.69512195 0.88942575 0.94830227 0.96936882 0.93841174
 0.87207146 0.93817917 0.83760129 0.87657672 0.91298477 0.97135392
 0.73140813 0.94941331 0.96656937], fscore: [0.93132474 0.67713706 0.92691103 0.93061963 0.95653929 0.94552984
 0.92395435 0.9464939  0.85778224 0.9278165  0.93067553 0.96696855
 0.83369284 0.96267238 0.9

epoch49, iter50, loss: 0.0006602155626751482
epoch49, iter60, loss: 0.0009200692875310779
epoch49, iter70, loss: 0.00019030450494028628
epoch49, iter80, loss: 0.0012625225353986025
epoch49, iter90, loss: 0.0006762693519704044
epoch49, iter100, loss: 0.0001964512630365789
epoch49, iter110, loss: 0.00019570357108023018
epoch49, iter120, loss: 0.0003392819780856371
epoch49, iter130, loss: 0.0002581305743660778
epoch49, iter140, loss: 0.0002632957184687257
epoch49, iter150, loss: 0.0002587687922641635
epoch49, iter160, loss: 0.0002910625480581075
epoch49, iter170, loss: 0.00034566267277114093
epoch49, iter180, loss: 0.00026008946588262916
epoch49, iter190, loss: 0.00027421925915405154
epoch49, iter200, loss: 0.0006897112471051514
epoch49, iter210, loss: 0.001301199896261096
epoch49, iter220, loss: 0.000480084796436131
epoch49, iter230, loss: 0.00036562979221343994
epoch49, iter240, loss: 0.0002798319037538022
epoch49, iter250, loss: 0.00036981605808250606
epoch49, iter260, loss: 0.00036364

## Prediction Result 

In [8]:
def prediction(model_name):
    
    # load pretrain models
    state_dict = torch.load(os.path.join(model_dir, model_name))
    fcn_model.load_state_dict(state_dict)
    
    batch = dataiter.next()
    if use_gpu:
        inputs = Variable(batch['X'].cuda())
    else:
        inputs = Variable(batch['X'])
    img    = batch['origin'] 
    label  = batch['l']
    output = fcn_model(inputs)
    output = output.data.cpu().numpy()

    N, _, h, w = output.shape
    pred = output.transpose(0, 2, 3, 1).reshape(-1, n_class).argmax(axis = 1).reshape(N, h, w)

    # show images
    plt.figure(figsize = (10, 12))
    img = img.numpy()
    for i in range(N):
        img[i] = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)
        plt.subplot(N, 3, i*3 + 1)
        plt.title("origin_img")
        plt.imshow(img[i])
        print(np.unique(label[i]))

        plt.subplot(N, 3, i*3 + 2)
        plt.title("label_img")
        plt.imshow(label[i],cmap = "gray",vmin = 0, vmax = n_class - 1)
        print(np.unique(pred[i]))
        plt.subplot(N, 3, i*3 + 3)
        plt.title("prediction")
        plt.imshow(pred[i],cmap = "gray",vmin = 0, vmax = n_class - 1)

    plt.show()

In [ ]:
prediction("FCNs_text_spotting_batch6_epoch49_RMSprop_lr0.0001.pkl")